## **Challenge:** <i><font color="red">Challenge description</font></i>

### Content:
* Dataset itroduction
* Assumptions
* Questions
* Exploratory Data Analysis
* Dealing with class imbalance
* Training and testing of various models performance
* Comclusions



____________________
## Dataset introduction

The data is originally from a [url description](https://www.kaggle.com/mlg-ulb/creditcardfraud).
Downloaded from [here](https://drive.google.com/file/d/1FCQY1SiWIjh_ME6Wtb3FG8Y1sKoRwAUc/view?usp=sharing). 

<i><font color="red"></font></i><i><font color="red">Introduction</font></i>

____________________
## Assumptions

Asssumptions

____________________
## Questions

Questions

## Exploratory Data Analysis

In [32]:
#Importing required Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In this challenge, we will try to predict credit card fraud.

Download the data from [here](https://drive.google.com/file/d/1FCQY1SiWIjh_ME6Wtb3FG8Y1sKoRwAUc/view?usp=sharing). The data is originally from a [Kaggle Competition](https://www.kaggle.com/mlg-ulb/creditcardfraud).

In [8]:
import psycopg2
from psycopg2 import Error

try:
    # Connect to an existing database
    connection = psycopg2.connect(user="lhl_student",
                                  password="lhl_student",
                                  host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com",
                                  port="5432",
                                  database="mid_term_project")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT * from flights limit 100")
    #list_tables = cursor.fetchall()
    #print(list_tables)
    
    # Fetch result
    #list_tables = cursor.fetchall()
    #print(list_tables)
    #record = cursor.fetchnone()
    #print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL server information
{'user': 'lhl_student', 'channel_binding': 'prefer', 'dbname': 'mid_term_project', 'host': 'mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

PostgreSQL connection is closed


In [11]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df
    df.head()